In [1]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import urllib
%matplotlib inline
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time


In [2]:

# Setup the ChromeDriver with a valid User-Agent
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Optional, to run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# Set a valid User-Agent header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=options)

headers = {
    'Accept-Language': 'en',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'X-FORWARDED-FOR': '2.21.184.0'
}

In [36]:
pmg = 'https://pmg.org.za/question_replies/'

In [37]:
response = requests.get(pmg, headers=headers)
print(response.status_code)

200


In [38]:
pmg_soup = BeautifulSoup(response.text, 'html.parser')

In [39]:
def safe_extract(soup, selector, attribute, default=np.nan):
        """Helper function to extract text from a BeautifulSoup tag safely."""
        element = soup.find(selector, attribute)
        return element.get_text(strip=True) if element else default


# Helper function to extract text from multiple <p> tags within a specific div
def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
    """Helper function to extract text from multiple <p> tags safely inside a parent container."""
    parent_element = soup.find(parent_selector, parent_attribute)
    if parent_element:
        child_elements = parent_element.find_all(child_selector)
        return [element.get_text(strip=True) for element in child_elements]
    return []



In [40]:
# Extract the required fields
pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extracts the date and NW552 reference
pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-heading'}, default=np.nan)  # Extract the name and credentials of the person asking
pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question text

# Extract the content from all <p> tags within the specific 'committee-question-reply' div
pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

# Join paragraphs into one string for better readability
pmg_answer = " ".join(pmg_answer_paragraphs)



# Output the extracted values
print("Date:", pmg_date)
print("Credentials:", pmg_credentials)
print("Question:", pm_question)
print("Answer:", pmg_answer)

Date: 04 October 2024 -NW1024
Credentials: Tetyana, Mr Yto ask the
Minister of Tourism
Question: What (a) urgent steps have been taken to revive (i) rural and (ii) township tourism in the Republic and (b) key (i) strategies and (ii) initiatives are in place that will be implemented to support and promote tourism in the specified areas?
Answer: (a)(i) and (ii) What urgent steps have been taken to revive (i) rural tourism and (ii) and township tourism in the Republic (b) What key (i) strategies are in place that will be implemented to support and promote tourism in the specified areas? The strategy involves increasing geographic spread of visitors, the diversification of products and inclusive participation by communities. (b) What (ii) initiatives are in place that will be implemented to support and promote tourism in the specified areas? I am further informed by South African Tourism that In support of the rural and township areas, they have implemented the following programmes: The Na

In [53]:
def fetch_soup(pmg_url, headers):
    """
    Fetch the content of a URL (base URL + movie link) and return a BeautifulSoup object.

    Args:
        base_url (str): The base URL (e.g., 'https://www.imdb.com/').
        movie_link (str): The relative URL or path to the movie page.
        headers (dict): Headers to be used in the HTTP request.

    Returns:
        BeautifulSoup: Parsed HTML content of the page if successful, else None.
    """
    

    # Fetch the page content
    response = requests.get(pmg_url, headers=headers)
    
    # Check for a successful response and return BeautifulSoup object
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    
    return None


In [60]:
def get_pmg_content(pmg_soup):
    # Helper function to extract text from a single tag
    def safe_extract(soup, selector, attribute, default=np.nan):
        """Helper function to extract text from a BeautifulSoup tag safely."""
        element = soup.find(selector, attribute)
        return element.get_text(strip=True) if element else default

    # Helper function to extract text from multiple <p> tags within a specific div
    def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
        """Helper function to extract text from multiple <p> tags safely inside a parent container."""
        parent_element = soup.find(parent_selector, parent_attribute)
        if parent_element:
            child_elements = parent_element.find_all(child_selector)
            return [element.get_text(strip=True) for element in child_elements]
        return []

    # Extract the required fields from 'committee-question-detail'
    pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extract the date and NW552 reference from the <h4> tag
    pmg_credentials = safe_extract(pmg_soup, 'div', {'class': 'media-body'}, default=np.nan)  # Extract name and credentials of the person asking from 'media-body' class
    pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question from the first <p> tag in the 'committee-question-detail' div

    # Extract the content from all <p> tags within the 'committee-question-reply' div
    pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

    # Join paragraphs into one string for better readability
    pmg_answer = " ".join(pmg_answer_paragraphs)

    return pmg_date, pmg_credentials, pm_question, pmg_answer

In [63]:

def get_all_pmg_content(pages=5, headers=headers, verbose=False):
    columns = ['date', 'mp', 'question', 'answer']
    data = []
    done = 0
    base_url = 'https://pmg.org.za/question_replies/'

    start = time.time()
    
    for page in range(1, pages):
        
        soup = fetch_soup(f'{base_url}{page}', headers)
        
        if soup:
            pages_stat = list(get_pmg_content(soup))
            # data.append([page] + pages_stat)
            data.append(pages_stat)
        else:
            # data.append([page] + [np.nan] * 4)
            data.append([np.nan] * 4)
        
        if verbose:
            done += 1
            print(f'Done with {done} pages. {pages - done} left. Elapsed time: {(time.time() - start):.1f} seconds')
        
        time.sleep(1)  # To avoid overwhelming the server
    
    return pd.DataFrame(data, columns=columns)
                  


In [67]:
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Assuming fetch_soup and get_pmg_content are defined somewhere else
# fetch_soup fetches the HTML content and returns a BeautifulSoup object
# get_pmg_content extracts the date, MP, question, and answer from a single element

def get_all_pmg_content(pages=5, headers=None, verbose=False):
    columns = ['date', 'mp', 'question', 'answer']
    data = []
    done = 0
    base_url = 'https://pmg.org.za/question_replies/'

    start = time.time()

    for page in range(1, pages + 1):  # Loop through all specified pages
        soup = fetch_soup(f'{base_url}{page}', headers)  # Fetch soup for each page

        if soup:
            # Find all 'committee-question-detail' elements on the page
            question_details = soup.select('article > section .committee-question-detail')

            # For each 'committee-question-detail' element, scrape the content
            for detail in question_details:
                content = get_pmg_content(detail)  # Extract the content for each detail
                data.append(content)  # Append the extracted content to the data list
        else:
            # If no content is found, append NaNs to maintain consistent rows
            data.append([np.nan] * 4)

        # Print verbose progress
        if verbose:
            done += 1
            print(f'Done with {done} pages. {pages - done} left. Elapsed time: {(time.time() - start):.1f} seconds')

        time.sleep(1)  # Pause to avoid overwhelming the server

    # Return the scraped data as a DataFrame
    return pd.DataFrame(data, columns=columns)



In [68]:
get_all_pmg_content(pages=5, headers=headers, verbose=1)

Done with 1 pages. 4 left. Elapsed time: 5.4 seconds
Done with 2 pages. 3 left. Elapsed time: 8.4 seconds
Done with 3 pages. 2 left. Elapsed time: 11.6 seconds
Done with 4 pages. 1 left. Elapsed time: 14.5 seconds
Done with 5 pages. 0 left. Elapsed time: 17.0 seconds


,date,mp,question,answer
0,04 October 2024 -NW1024,"Tetyana, Mr Yto ask the\nMinister of Tourism",What (a) urgent steps have been taken to reviv...,(a)(i) and (ii) What urgent steps have been ta...
1,04 October 2024 -NW1054,"Clarke, Ms Mto ask the\nMinister of Health",(1) What is the total number of (a) persons (i...,(1) (a) (i) Total number of patients on waitin...
2,04 October 2024 -NW974,"Maimane, Mr MAto ask the\nMinister of Tourism",What is the (a) total number of staff employed...,(i) MINISTER (a)(i) Total number of staff empl...
3,04 October 2024 -NW967,"Madlala, Mr KBto ask the\nMinister of Water an...",(1) Given that ward 45 under the eThekwini Met...,MINISTER OF WATER AND SANITATION (1) The eThek...
4,04 October 2024 -NW976,"Webster, Ms NLto ask the\nMinister of Transport",What is the (a) total number of staff employed...,Find replyhere
...,...,...,...,...
245,18 September 2024 -NW787,"Mncwango, Mr MAto ask the\nMineral and Petrole...",In light of the fact that TotalEnergies South ...,(a) The oil and gas sector is regulated by the...
246,18 September 2024 -NW535,"Engelbrecht, Mr Jto ask the\nMinister of Corre...",What is the total number of (a) section 62(F) ...,Find replyhere
247,18 September 2024 -NW832,"Petersen, Ms JSto ask the\nMinister of Mineral...",What is the latest progress of his department ...,The prevailing “litigious” environment related...
248,18 September 2024 -NW812,"Ntlangwini, Ms ENto ask the\nMinister of Miner...",What (a) are the full details of active and po...,South Africa is potentially endowed with oil a...
